In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


def weight_tern(fp_w, t_factor):
    fp_w = torch.from_numpy(fp_w)
    mean_w = fp_w.abs().mean()
    max_w = fp_w.abs().max()
    th = t_factor*max_w

    output = fp_w.clone().zero_()
    W = fp_w[fp_w.ge(th)+fp_w.le(-th)].abs().mean()
    output[fp_w.ge(th)] = 1
    output[fp_w.lt(-th)] = -1
#     print('W:{}'.format(W))
    return output.numpy()

model_dir = '/Users/jianmeng/Desktop/ASU_research/analysis/exp_results/imagenet_test/resnet18_w4_a4_swpTrue_from_quant/checkpoint_{epoch}.pth.tar'
# model = torch.load(model_dir, map_location=torch.device('cpu'))
model = torch.load(model_dir)
params = model['state_dict']

counter = 0
overall = 0
num_one = 0
all_num_one = 0.0
all_num_group = 0.0
group_ch = 16
all_sparsity_mean = 0.0
all_sparsity_min = 1.0
all_num = 0.0
count_num_one = 0.0
t_factor = 0.05
tern = True

for k,v in params.items():
    if 'stage' in k and 'conv' in k:
        print(k)
        
        counter += 1
        conv_w = v 
        output = np.array(conv_w)
        cout = output.shape[0]
        cin = output.shape[1]
        kh = output.shape[2]
        kw = output.shape[3]
        
        if tern:
            out_sparse = weight_tern(output, t_factor)
        else:
        # == set threshold for weight
            out_sparse = np.zeros(output.shape)
            out_sparse[np.absolute(output) > 1e-3] = output[np.absolute(output) > 1e-3]
        # ==
        
        #cal the whole number of weight in current layer
        count_num_layer = cout * cin * kh * kw
        all_num += count_num_layer
        #cal the nonzero weight in current layer
        count_one_layer = np.count_nonzero(out_sparse)
        count_num_one += count_one_layer
        
        w_t = out_sparse.reshape(cout, cin // group_ch, group_ch, kh, kw)
        num_group = (cout * cin) // group_ch
        w_t = w_t.reshape(num_group, group_ch, kh, kw)
        print(f"{k}, The shape of w_t: {w_t.shape}")
        all_num_group += num_group
        sparsity_chall = np.zeros(num_group)
        num_one = 0
        for i in range(num_group):       
            sparsity_group = 1.0*(w_t[i].size - np.count_nonzero(w_t[i]))/w_t[i].size
            sparsity_chall[i] = sparsity_group
            if np.count_nonzero(w_t[i]) is 0:
                num_one += 1
        all_num_one += num_one 
        print('num of sparse group:{}'.format(num_one))
        print('num of group:{}'.format(num_group))

print("==== conclusion ==== ")
overall_sparsity = 1 - count_num_one/all_num
print('overall sparsity for all layer:{:.6f}'.format(overall_sparsity))
group_sparsity = all_num_one/all_num_group
print('group sparsity for all layer:{:.6f}'.format(group_sparsity))

==== conclusion ==== 


ZeroDivisionError: float division by zero

In [ ]:
a = np.random.randint(5, size=(2, 4))
b = np.count_nonzero(a)
print(b)